In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time, TimezoneInfo, TimeDelta
from statsmodels.tsa.stattools import adfuller

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient, clipDataToEvent, calcNextDay
from lsst_efd_client import EfdClient
from tqdm import tqdm
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### For each slew:
 1. start and end position elevation/azimuth
 2. check if lsst.sal.MTM1M3.appliedVelocityForces and lsst.sal.MTM1M3.appliedAccelerationForces are zero 
 3. get velocity configuration from maxCmdVelocity/maxMoveVelocity

 

In [3]:
def get_data(begin, end, client):
    "Extract all the MTMount data from the EFD and add to dict"
    query_dict={}

    query_dict["el"] = getEfdData(
                client,
                "lsst.sal.MTMount.elevation",
                columns=["private_sndStamp", "private_efdStamp", "actualPosition", "actualVelocity", "actualTorque"],
                begin=begin,
                end=end,
                prePadding=10,
                postPadding=10,
                warn=False,
            )
    query_dict["az"] = getEfdData(
                client,
                "lsst.sal.MTMount.azimuth",
                columns=["private_sndStamp", "private_efdStamp", "actualPosition", "actualVelocity", "actualTorque"],
                begin=begin,
                end=end,
                prePadding=10,
                postPadding=10,
                warn=False,
            )
    return query_dict

def check_for_forces(event, client):
    appliedVelocityForces = getEfdData(
                client,
                "lsst.sal.MTM1M3.appliedVelocityForces",
                #columns=["private_sndStamp", ""],
                event=event,
                prePadding=0,
                postPadding=0,
                warn=False,
            )
    appliedAccelerationForces = getEfdData(
                client,
                "lsst.sal.MTM1M3.appliedAccelerationForces",
                #columns=["private_sndStamp", ""],
                event=event,
                prePadding=0,
                postPadding=0,
                warn=False,
            )
    
    
    ret_bool = appliedVelocityForces[[c for c in appliedVelocityForces.columns if "Forces" in c]].eq(0).all().eq(True).all()
    ret_bool &= appliedAccelerationForces[[c for c in appliedAccelerationForces.columns if "Forces" in c]].eq(0).all().eq(True).all()
    return ret_bool

In [4]:
state_dict={TMAState.TRACKING:"tracking",TMAState.SLEWING:"slewing"}
def characterize_slews(dayObs, client):
    # Select data from a given date
    eventMaker = TMAEventMaker()
    events = eventMaker.getEvents(dayObs)

    # Get lists of slew and track events
    events_list = [e for e in events if (e.type==TMAState.SLEWING) | (e.type==TMAState.TRACKING)]
    print(len(events_list))
    event_dict={
        "day_obs":[],
        "seq_num":[],
        "state":[],
        "time_begin":[],
        "time_end":[],
        "duration":[],
        "az_begin":[],
        "az_end":[],
        "az_max_vel":[],
        "el_begin":[],
        "el_end":[],
        "el_max_vel":[],
        "inertial_compensation":[],
        
    }
    if len(events_list) < 1:
        return event_dict
    query_dict= get_data(events_list[0].begin,events_list[-1].end, client)
    print("data queried")
    
    for event in tqdm(events_list):
        event_dict["day_obs"].append(dayObs)
        event_dict["seq_num"].append(event.seqNum)
        event_dict["state"].append(state_dict[event.type])
        event_dict["time_begin"].append(event.begin.unix)
        event_dict["time_end"].append(event.end.unix)
        event_dict["duration"].append(event.duration)
        for axis in ["az","el"]:
            subdat=clipDataToEvent(query_dict[axis], event)
            if len(subdat) < 2:
                event_dict[f"{axis}_begin"].append(np.nan) 
                event_dict[f"{axis}_end"].append(np.nan)
                event_dict[f"{axis}_max_vel"].append(np.nan)
            else:
                event_dict[f"{axis}_begin"].append(subdat["actualPosition"][0])
                event_dict[f"{axis}_end"].append(subdat["actualPosition"][-1])
                event_dict[f"{axis}_max_vel"].append(max(subdat["actualVelocity"], key=abs))
        event_dict["inertial_compensation"].append(check_for_forces(event, client)==False)
    return event_dict

In [8]:
begin_day_obs = 20230802
end_day_obs = 20230803

In [11]:
current_day_obs = begin_day_obs
while int(current_day_obs) <= int(end_day_obs):
    next_day_obs = calcNextDay(current_day_obs)
    dayObs=current_day_obs
    outpath=f"../data/{dayObs}_slew_characterization.csv"
    if os.path.exists(outpath):
        print(f"{dayObs} already_done")
    else: 
        print("hi")
    client = makeEfdClient()
    dat=characterize_slews(dayObs, client)
    summary_frame=pd.DataFrame(dat)
    summary_frame.to_csv(outpath)
        
    current_day_obs = next_day_obs

20230802 already_done


503
data queried


100%|██████████| 503/503 [07:36<00:00,  1.10it/s]


20230803 already_done


0
